In [1]:
!pip install gensim

In [2]:
!pip install keras

In [36]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.8 MB/s eta 0:00:00


In [37]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 10.1 MB/s eta 0:00:00


In [29]:
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import tensorflow
from tensorflow import keras
from keras import layers, Sequential, models, metrics

import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

import numpy as np
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path = '/content/drive/My Drive/sentimentDatasets/genshin_sentiment_cleaned.csv'
df = pd.read_csv(path)
df.dropna(inplace=True)
df

,Unnamed: 0,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion,cleaned_content,cleaned_lowercase,slangwords_content,tokenized_content,stopwords_content,final_content,score_polarity,polarity,content_length
0,4,713a4c4d-b537-402e-a919-7cf3e1cbb150,Pengguna Google,Menurutku ini udah bagus.,5,0,5.4.0_30057195_30231699,2025-04-07 02:30:26,5.4.0_30057195_30231699,Menurutku ini udah bagus,menurutku ini udah bagus,menurutku ini sudah bagus,"['menurutku', 'ini', 'sudah', 'bagus']","['menurutku', 'bagus']",menurutku bagus,2,positive,2
1,5,f0c79417-9334-4bf1-bca6-086c57240798,Pengguna Google,Minusnya resource seret.,3,0,5.4.0_30057195_30231699,2025-04-07 02:28:28,5.4.0_30057195_30231699,Minusnya resource seret,minusnya resource seret,minusnya resource seret,"['minusnya', 'resource', 'seret']","['minusnya', 'resource', 'seret']",minusnya resource seret,0,positive,3
2,6,f57f0768-3d07-48d3-b883-1c4a15f2a6c3,Pengguna Google,game nya bagus banyak yang suka sayangnya kada...,5,0,5.5.0_31400259_31451966,2025-04-07 02:13:02,5.5.0_31400259_31451966,game nya bagus banyak yang suka sayangnya kada...,game nya bagus banyak yang suka sayangnya kada...,game nya bagus banyak yang suka sayangnya kada...,"['game', 'nya', 'bagus', 'banyak', 'yang', 'su...","['game', 'bagus', 'suka', 'sayangnya', 'kadang...",game bagus suka sayangnya kadang kadang suka n...,21,positive,10
3,7,b604dfc7-8986-4562-96c6-86fc3fc0569d,Pengguna Google,seru banget,5,0,5.5.0_31400259_31451966,2025-04-07 01:17:09,5.5.0_31400259_31451966,seru banget,seru banget,seru banget,"['seru', 'banget']","['seru', 'banget']",seru banget,6,positive,2
4,8,367cf423-f24e-489c-848d-d6648b841952,Pengguna Google,"terimakasih sudah tidak kikir,semoga makin ban...",5,1,5.5.0_31400259_31451966,2025-04-07 01:12:54,5.5.0_31400259_31451966,terimakasih sudah tidak kikirsemoga makin bany...,terimakasih sudah tidak kikirsemoga makin bany...,terimakasih sudah tidak kikirsemoga makin bany...,"['terimakasih', 'sudah', 'tidak', 'kikirsemoga...","['terimakasih', 'kikirsemoga', 'bansos', 'graf...",terimakasih kikirsemoga bansos grafiknya keren...,10,positive,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139578,216037,3f7f8ab4-e7f7-42a5-a6d2-468c6bfa5cfc,Iskandar Zulkarnaen,Game Anike bagus,5,0,1.0.0_1112729_1135452,2020-09-27 04:35:41,1.0.0_1112729_1135452,Game Anike bagus,game anike bagus,game anike bagus,"['game', 'anike', 'bagus']","['game', 'anike', 'bagus']",game anike bagus,4,positive,3
139579,216042,68cabe23-14bc-4771-be24-85f0035625fb,Muhammad Abyan Tsabit Amani,Game nya sangat mantap sekali,5,2,1.0.0_1112729_1135452,2020-09-27 04:31:21,1.0.0_1112729_1135452,Game nya sangat mantap sekali,game nya sangat mantap sekali,game nya sangat mantap sekali,"['game', 'nya', 'sangat', 'mantap', 'sekali']","['game', 'mantap']",game mantap,7,positive,2
139580,216048,a697bff7-22e3-4ce0-a509-5a40d5781663,Akunnah Cobaan,Last go!!,5,2,1.0.0_1112729_1135452,2020-09-27 04:28:29,1.0.0_1112729_1135452,Last go,last go,last go,"['last', 'go']","['last', 'go']",last go,0,positive,2
139581,216051,08c3b047-a6a8-463b-80a9-db212e8c8716,Edwin Jhon,"Mihoyo, Otaku technologi Save the world 😎🔥",5,1,1.0.0_1112729_1135452,2020-09-27 04:28:07,1.0.0_1112729_1135452,Mihoyo Otaku technologi Save the world emoji e...,mihoyo otaku technologi save the world emoji e...,mihoyo otaku technologi save the world emoji e...,"['mihoyo', 'otaku', 'technologi', 'save', 'the...","['mihoyo', 'otaku', 'technologi', 'save', 'wor...",mihoyo otaku technologi save world emoji emoji,0,positive,7


In [6]:
x = df['final_content']
y = df['polarity']

In [7]:
#tfid
tfidf = TfidfVectorizer(max_features =100, min_df=20, max_df=0.8)
x_tfidf = tfidf.fit_transform(x)

tfidf_df = pd.DataFrame(x_tfidf.toarray(), columns=tfidf.get_feature_names_out())
tfidf_df

,akun,alur,ampas,android,anniversary,bagus,banget,berat,bermain,best,...,suka,susah,teman,terbaik,terima,terimakasih,tolong,udh,update,world
0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.246731,0.000000,0.000000,0.0,0.0,...,0.813321,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.677491,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.242485,0.160880,0.255529,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.259808,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138816,0.0,0.0,0.0,0.0,0.0,0.772959,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
138817,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
138818,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
138819,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.579306


In [8]:
#wor2vec
tokenized = [word_tokenize(sentence) for sentence in x]
wordmodel = Word2Vec(sentences=tokenized, vector_size=100, window=5, min_count=20, workers=4)

In [9]:
def document_vector(doc):
    doc = [word for word in doc if word in wordmodel.wv]
    if len(doc) == 0:
        return np.zeros(wordmodel.vector_size)
    return np.mean(wordmodel.wv[doc], axis=0)
doc_vectors = [document_vector(doc) for doc in tokenized]
w2v_df = pd.DataFrame(doc_vectors)
w2v_df.columns = [f'w2v_{i}' for i in range(wordmodel.vector_size)]
w2v_df

,w2v_0,w2v_1,w2v_2,w2v_3,w2v_4,w2v_5,w2v_6,w2v_7,w2v_8,w2v_9,...,w2v_90,w2v_91,w2v_92,w2v_93,w2v_94,w2v_95,w2v_96,w2v_97,w2v_98,w2v_99
0,0.319256,0.329793,0.276505,-0.619867,0.087179,0.214120,0.205188,0.034874,-0.203206,-0.073162,...,0.551777,0.722184,0.320215,0.095246,0.600608,0.188909,-0.100238,0.278816,-0.071320,-0.555941
1,0.165504,0.174390,0.066616,0.110036,-0.203394,-0.098783,0.040492,0.190075,0.074658,0.023646,...,0.115229,0.087696,-0.060884,0.113887,0.131769,0.089293,0.044400,0.080836,-0.192748,-0.161246
2,0.294502,-0.145890,0.171818,-0.515842,0.216935,-0.231585,-0.207930,0.168651,-0.658250,0.181342,...,0.070610,0.383604,0.341751,0.023521,0.681078,0.338513,0.078774,-0.018437,0.328784,-0.942525
3,0.244816,0.486703,0.168834,-0.880793,0.310868,-0.061426,0.450935,0.424445,-0.127084,0.229840,...,0.328051,0.424624,0.472368,0.065071,0.713240,0.531105,0.161771,0.097197,-0.301915,-1.199310
4,0.216699,-0.024456,0.151759,-0.429803,-0.081600,-0.011733,0.059624,-0.098849,0.189243,0.283190,...,0.185135,0.374008,0.071695,0.055603,0.411996,0.180704,-0.067273,0.046911,-0.059080,-0.273886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138816,0.429705,0.270697,0.880097,-0.245123,-0.146706,-0.249302,0.392754,0.042609,-0.481961,-0.361516,...,0.406154,0.649032,0.579457,0.312281,0.571027,0.121602,-0.134668,0.060275,0.065856,-0.641609
138817,-0.018962,0.342817,0.734238,-0.129724,-0.021793,-0.541219,0.568561,0.160445,-0.400541,-0.099101,...,0.253026,0.540321,0.644656,0.246759,0.462019,0.004333,0.147661,0.134236,0.008946,-0.817520
138818,0.072680,0.110540,-0.304123,0.059855,-0.437006,0.056839,0.402578,0.573962,0.001398,0.192060,...,0.376819,-0.210261,-0.309647,0.104336,0.132771,-0.057336,0.096709,-0.166303,0.043517,0.183755
138819,0.160949,-0.102157,-0.265242,-0.492699,0.107029,-0.656287,0.321007,0.406544,-0.299137,0.311796,...,0.480945,0.327640,0.177103,0.026329,0.163648,-0.435046,0.455424,0.575382,-0.104773,-0.334636


In [10]:
#splitting
x_temp_tfidf, x_test_tfidf, y_temp_tfidf, y_test_tfidf = train_test_split(tfidf_df, y, test_size=0.2, random_state=42)
x_temp_w2v, x_test_w2v, y_temp_w2v, y_test_w2v = train_test_split(w2v_df, y, test_size=0.2, random_state=42)

x_train_tfidf, x_val_tfidf, y_train_tfidf, y_val_tfidf = train_test_split(x_temp_tfidf, y_temp_tfidf, test_size=0.25, random_state=42)
x_train_w2v, x_val_w2v, y_train_w2v, y_val_w2v = train_test_split(x_temp_w2v, y_temp_w2v, test_size=0.25, random_state=42)

In [11]:
#model RF - w2v
rf_model = RandomForestClassifier(n_estimators=250, max_depth=10,
                            min_samples_leaf=4)
rf_model.fit(x_train_w2v, y_train_w2v)

pred_w2v_rf = rf_model.predict(x_test_w2v)
train_w2v_rf = rf_model.predict(x_train_w2v)

acc_train_w2v_rf = accuracy_score(y_train_w2v, train_w2v_rf)
acc_test_w2v_rf = accuracy_score(y_test_w2v, pred_w2v_rf)

print(f'Train accuracy: {acc_train_w2v_rf}')
print(f'Test accuracy: {acc_test_w2v_rf}')

Train accuracy: 0.9186236373241128
Test accuracy: 0.909742481541509


In [17]:
#model RF - TFIDF
rf_model = RandomForestClassifier(n_estimators=250, max_depth=10,
                            min_samples_leaf=4)
rf_model.fit(x_train_tfidf, y_train_tfidf)

pred_tfidf_rf = rf_model.predict(x_test_tfidf)
train_tfidf_rf = rf_model.predict(x_train_tfidf)

acc_train_tfidf_rf = accuracy_score(y_train_tfidf, train_tfidf_rf)
acc_test_tfidf_rf = accuracy_score(y_test_tfidf, pred_tfidf_rf)

print(f'Train acc : {acc_train_tfidf_rf}')
print(f'Test acc : {acc_test_tfidf_rf}')

Train acc : 0.8999903952360371
Test acc : 0.9004862236628849


In [20]:
#model LSTM-w2v
le = LabelEncoder()
y_train_w2v = le.fit_transform(y_train_w2v)
y_val_w2v = le.transform(y_val_w2v)
y_test_w2v = le.transform(y_test_w2v)

def model():
  model = models.Sequential([
      layers.LSTM(256, input_shape=(x_train_w2v.shape[1], 1), return_sequences = True),
      layers.Dropout(0.3),
      layers.LSTM(128, return_sequences = True),
      layers.Dropout(0.2),
      layers.LSTM(64, return_sequences = True),
      layers.Dropout(0.1),
      layers.LSTM(32, return_sequences = True),
      layers.LSTM(16, return_sequences = False),
      layers.Dense(1, activation='sigmoid')
  ])
  optimizer = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

model = model()
model.fit(x_train_w2v, y_train_w2v, epochs=25, batch_size=64, validation_data=(x_val_w2v, y_val_w2v))
pred_w2v_lstm =( model.predict(x_test_w2v) > 0.5).astype("int32")
train_w2v_lstm = (model.predict(x_train_w2v) > 0.5).astype("int32")

acc_train_w2v_lstm = accuracy_score(y_train_w2v, train_w2v_lstm)
acc_test_w2v_lstm = accuracy_score(y_test_w2v, pred_w2v_lstm)

print(f'Train acc : {acc_train_w2v_lstm}')
print(f'Test acc : {acc_test_w2v_lstm}')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 65s 47ms/step - accuracy: 0.8885 - loss: 0.3516 - val_accuracy: 0.8923 - val_loss: 0.3398
Epoch 2/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 75s 41ms/step - accuracy: 0.8931 - loss: 0.3385 - val_accuracy: 0.8923 - val_loss: 0.3404
Epoch 3/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 73s 35ms/step - accuracy: 0.8890 - loss: 0.3472 - val_accuracy: 0.8923 - val_loss: 0.3403
Epoch 4/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 95s 45ms/step - accuracy: 0.8919 - loss: 0.3414 - val_accuracy: 0.8923 - val_loss: 0.3361
Epoch 5/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 62s 30ms/step - accuracy: 0.8907 - loss: 0.3256 - val_accuracy: 0.8923 - val_loss: 0.3121
Epoch 6/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 42s 30ms/step - accuracy: 0.8930 - loss: 0.3071 - val_accuracy: 0.8923 - val_loss: 0.2898
Epoch 7/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 42s 31ms/step - accuracy: 0.8923 - loss: 0.2944 - val_accuracy: 0.8923 - val_loss: 0.3417
Epoch 8/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.8904 -

In [13]:
#model LSTM-tfidf
le = LabelEncoder()
y_train_tfidf = le.fit_transform(y_train_tfidf)
y_val_tfidf = le.transform(y_val_tfidf)
y_test_tfidf = le.transform(y_test_tfidf)

def model():
  model = models.Sequential([
      layers.LSTM(256, input_shape=(x_train_tfidf.shape[1], 1), return_sequences = True),
      layers.Dropout(0.3),
      layers.LSTM(128, return_sequences = True),
      layers.Dropout(0.2),
      layers.LSTM(64, return_sequences = True),
      layers.Dropout(0.1),
      layers.LSTM(32, return_sequences = True),
      layers.LSTM(16, return_sequences = False),
      layers.Dense(1, activation='sigmoid')
  ])
  optimizer = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

model = model()
model.fit(x_train_tfidf, y_train_tfidf, epochs=25, batch_size=64, validation_data=(x_val_tfidf, y_val_tfidf))
pred_tfidf_lstm =( model.predict(x_test_tfidf) > 0.5).astype("int32")
train_tfidf_lstm = (model.predict(x_train_tfidf) > 0.5).astype("int32")

acc_train_tfidf_lstm = accuracy_score(y_train_tfidf, train_tfidf_lstm)
acc_test_tfidf_lstm = accuracy_score(y_test_tfidf, pred_tfidf_lstm)

print(f'Train acc : {acc_train_tfidf_lstm}')
print(f'Test acc : {acc_test_tfidf_lstm}')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 55s 31ms/step - accuracy: 0.8883 - loss: 0.3534 - val_accuracy: 0.8923 - val_loss: 0.3423
Epoch 2/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 38s 29ms/step - accuracy: 0.8893 - loss: 0.3483 - val_accuracy: 0.8923 - val_loss: 0.3417
Epoch 3/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 43s 31ms/step - accuracy: 0.8922 - loss: 0.3421 - val_accuracy: 0.8923 - val_loss: 0.3417
Epoch 4/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 40s 31ms/step - accuracy: 0.8910 - loss: 0.3447 - val_accuracy: 0.8923 - val_loss: 0.3419
Epoch 5/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 40s 30ms/step - accuracy: 0.8924 - loss: 0.3415 - val_accuracy: 0.8923 - val_loss: 0.3422
Epoch 6/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 41s 30ms/step - accuracy: 0.8910 - loss: 0.3447 - val_accuracy: 0.8923 - val_loss: 0.3418
Epoch 7/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 41s 30ms/step - accuracy: 0.8904 - loss: 0.3459 - val_accuracy: 0.8923 - val_loss: 0.3420
Epoch 8/25
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 41s 30ms/step - accuracy: 0.8912 -

In [44]:
#resize split - RF - w2v
#splitting
x_temp_w2v, x_test_resize, y_temp_w2v, y_test_resize = train_test_split(w2v_df, y, test_size=0.3, random_state=42)
x_train_resize, x_val_w2v, y_train_resize, y_val_w2v = train_test_split(x_temp_w2v, y_temp_w2v, test_size=0.25, random_state=42)

rf_model_w2v = RandomForestClassifier(n_estimators=250, max_depth=10,
                            min_samples_leaf=3)
rf_model_w2v.fit(x_train_resize, y_train_resize)

pred_w2v_resize = rf_model.predict(x_test_resize)
train_w2v_resize = rf_model.predict(x_train_resize)

acc_train_w2v_resize = accuracy_score(y_train_resize, train_w2v_resize)
acc_test_w2v_resize = accuracy_score(y_test_resize, pred_w2v_resize)

print(f'Train accuracy: {acc_train_w2v_resize}')
print(f'Test accuracy: {acc_test_w2v_resize}')

Train accuracy: 0.9193605927552141
Test accuracy: 0.9117343386078229


In [15]:
#resize split - RF - TFIDF
x_temp_tfidf, x_test_resize, y_temp_tfidf, y_test_resize = train_test_split(tfidf_df, y, test_size=0.3, random_state=42)
x_train_resize, x_val_resize, y_train_resize, y_val_resize = train_test_split(x_temp_tfidf, y_temp_tfidf, test_size=0.25, random_state=42)

rf_model = RandomForestClassifier(n_estimators=250, max_depth=10,
                            min_samples_leaf=3)
rf_model.fit(x_train_resize, y_train_resize)

pred_tfidf_resize = rf_model.predict(x_test_resize)
train_tfidf_resize = rf_model.predict(x_train_resize)

acc_train_tfidf_resize = accuracy_score(y_train_resize, train_tfidf_resize)
acc_test_tfidf_resize = accuracy_score(y_test_resize, pred_tfidf_resize)

print(f'Train accuracy: {acc_train_tfidf_resize}')
print(f'Test accuracy: {acc_test_tfidf_resize}')

Train accuracy: 0.9006997804610318
Test accuracy: 0.9013374312675583


In [50]:
arr_train = np.round(np.array([
    acc_train_w2v_rf, acc_train_tfidf_rf, acc_train_w2v_lstm,
    acc_train_tfidf_lstm, acc_train_w2v_resize, acc_train_tfidf_resize
]), 3)

arr_test = np.round(np.array([
    acc_test_w2v_rf, acc_test_tfidf_rf, acc_test_w2v_lstm,
    acc_test_tfidf_lstm, acc_test_w2v_resize, acc_test_tfidf_resize
]), 3)

col = np.array([
    'Random Forest - w2v','Random Forest - TFIDF','LSTM - w2v','LSTM - TFIDF',
    'Random Forest - w2v - Resized Split','Random Forest - TFIDF - Resized Split'
])


In [51]:
resul = pd.DataFrame({'Train': arr_train, 'Test': arr_test}, index=col)
resul

,Train,Test
Random Forest - w2v,0.919,0.910
Random Forest - TFIDF,0.900,0.900
LSTM - w2v,0.911,0.907
LSTM - TFIDF,0.891,0.892
Random Forest - w2v - Resized Split,0.919,0.912
Random Forest - TFIDF - Resized Split,0.901,0.901


Berdasarkan data diatas, dapat disimpulkan bahwa model yang menggunakan feature extraction w2v memiliki hasil yang lebih baik dibandingkan menggunakan TFIDF, dimana hasil dengan kedua jenis model cukup dekat tetapi RF memiliki hasil yang lebih memuaskan

Untuk splitting, hasil kedua yang resized memiliki hasil yang lebih memuaskan.

In [38]:
# @title
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal", "plisss": "tolong", "bgttt": "banget", "indo": "indonesia", "bgtt": "banget", "ad": "ada", "rv": "redvelvet", "plis": "tolong", "pls": "tolong", "cr": "sumber", "cod": "bayar ditempat", "adlh": "adalah", "afaik": "as far as i know", "ahaha": "haha", "aj": "saja", "ajep-ajep": "dunia gemerlap", "ak": "saya", "akika": "aku", "akkoh": "aku", "akuwh": "aku", "alay": "norak", "alow": "halo", "ambilin": "ambilkan", "ancur": "hancur", "anjrit": "anjing", "anter": "antar", "ap2": "apa-apa", "apasih": "apa sih", "apes": "sial", "aps": "apa", "aq": "saya", "aquwh": "aku", "asbun": "asal bunyi", "aseekk": "asyik", "asekk": "asyik", "asem": "asam", "aspal": "asli tetapi palsu", "astul": "asal tulis", "ato": "atau", "au ah": "tidak mau tahu", "awak": "saya", "ay": "sayang", "ayank": "sayang", "b4": "sebelum", "bakalan": "akan", "bandes": "bantuan desa", "bangedh": "banget", "banpol": "bantuan polisi", "banpur": "bantuan tempur", "basbang": "basi", "bcanda": "bercanda", "bdg": "bandung", "begajulan": "nakal", "beliin": "belikan", "bencong": "banci", "bentar": "sebentar", "ber3": "bertiga", "beresin": "membereskan", "bete": "bosan", "beud": "banget", "bg": "abang", "bgmn": "bagaimana", "bgt": "banget", "bijimane": "bagaimana", "bintal": "bimbingan mental", "bkl": "akan", "bknnya": "bukannya", "blegug": "bodoh", "blh": "boleh", "bln": "bulan", "blum": "belum", "bnci": "benci", "bnran": "yang benar", "bodor": "lucu", "bokap": "ayah", "boker": "buang air besar", "bokis": "bohong", "boljug": "boleh juga", "bonek": "bocah nekat", "boyeh": "boleh", "br": "baru", "brg": "bareng", "bro": "saudara laki-laki", "bru": "baru", "bs": "bisa", "bsen": "bosan", "bt": "buat", "btw": "ngomong-ngomong", "buaya": "tidak setia", "bubbu": "tidur", "bubu": "tidur", "bumil": "ibu hamil", "bw": "bawa", "bwt": "buat", "byk": "banyak", "byrin": "bayarkan", "cabal": "sabar", "cadas": "keren", "calo": "makelar", "can": "belum", "capcus": "pergi", "caper": "cari perhatian", "ce": "cewek", "cekal": "cegah tangkal", "cemen": "penakut", "cengengesan": "tertawa", "cepet": "cepat", "cew": "cewek", "chuyunk": "sayang", "cimeng": "ganja", "cipika cipiki": "cium pipi kanan cium pipi kiri", "ciyh": "sih", "ckepp": "cakep", "ckp": "cakep", "cmiiw": "correct me if i'm wrong", "cmpur": "campur", "cong": "banci", "conlok": "cinta lokasi", "cowwyy": "maaf", "cp": "siapa", "cpe": "capek", "cppe": "capek", "cucok": "cocok", "cuex": "cuek", "cumi": "Cuma miscall", "cups": "culun", "curanmor": "pencurian kendaraan bermotor", "curcol": "curahan hati colongan", "cwek": "cewek", "cyin": "cinta", "d": "di", "dah": "deh", "dapet": "dapat", "de": "adik", "dek": "adik", "demen": "suka", "deyh": "deh", "dgn": "dengan", "diancurin": "dihancurkan", "dimaafin": "dimaafkan", "dimintak": "diminta", "disono": "di sana", "dket": "dekat", "dkk": "dan kawan-kawan", "dll": "dan lain-lain", "dlu": "dulu", "dngn": "dengan", "dodol": "bodoh", "doku": "uang", "dongs": "dong", "dpt": "dapat", "dri": "dari", "drmn": "darimana", "drtd": "dari tadi", "dst": "dan seterusnya", "dtg": "datang", "duh": "aduh", "duren": "durian", "ed": "edisi", "egp": "emang gue pikirin", "eke": "aku", "elu": "kamu", "emangnya": "memangnya", "emng": "memang", "endak": "tidak", "enggak": "tidak", "envy": "iri", "ex": "mantan", "fax": "facsimile", "fifo": "first in first out", "folbek": "follow back", "fyi": "sebagai informasi", "gaada": "tidak ada uang", "gag": "tidak", "gaje": "tidak jelas", "gak papa": "tidak apa-apa", "gan": "juragan", "gaptek": "gagap teknologi", "gatek": "gagap teknologi", "gawe": "kerja", "gbs": "tidak bisa", "gebetan": "orang yang disuka", "geje": "tidak jelas", "gepeng": "gelandangan dan pengemis", "ghiy": "lagi", "gile": "gila", "gimana": "bagaimana", "gino": "gigi nongol", "githu": "gitu", "gj": "tidak jelas", "gmana": "bagaimana", "gn": "begini", "goblok": "bodoh", "golput": "golongan putih", "gowes": "mengayuh sepeda", "gpny": "tidak punya", "gr": "gede rasa", "gretongan": "gratisan", "gtau": "tidak tahu", "gua": "saya", "guoblok": "goblok", "gw": "saya", "ha": "tertawa", "haha": "tertawa", "hallow": "halo", "hankam": "pertahanan dan keamanan", "hehe": "he", "helo": "halo", "hey": "hai", "hlm": "halaman", "hny": "hanya", "hoax": "isu bohong", "hr": "hari", "hrus": "harus", "hubdar": "perhubungan darat", "huff": "mengeluh", "hum": "rumah", "humz": "rumah", "ilang": "hilang", "ilfil": "tidak suka", "imho": "in my humble opinion", "imoetz": "imut", "item": "hitam", "itungan": "hitungan", "iye": "iya", "ja": "saja", "jadiin": "jadi", "jaim": "jaga image", "jayus": "tidak lucu", "jdi": "jadi", "jem": "jam", "jga": "juga", "jgnkan": "jangankan", "jir": "anjing", "jln": "jalan", "jomblo": "tidak punya pacar", "jubir": "juru bicara", "jutek": "galak", "k": "ke", "kab": "kabupaten", "kabor": "kabur", "kacrut": "kacau", "kadiv": "kepala divisi", "kagak": "tidak", "kalo": "kalau", "kampret": "sialan", "kamtibmas": "keamanan dan ketertiban masyarakat", "kamuwh": "kamu", "kanwil": "kantor wilayah", "karna": "karena", "kasubbag": "kepala subbagian", "katrok": "kampungan", "kayanya": "kayaknya", "kbr": "kabar", "kdu": "harus", "kec": "kecamatan", "kejurnas": "kejuaraan nasional", "kekeuh": "keras kepala", "kel": "kelurahan", "kemaren": "kemarin", "kepengen": "mau", "kepingin": "mau", "kepsek": "kepala sekolah", "kesbang": "kesatuan bangsa", "kesra": "kesejahteraan rakyat", "ketrima": "diterima", "kgiatan": "kegiatan", "kibul": "bohong", "kimpoi": "kawin", "kl": "kalau", "klianz": "kalian", "kloter": "kelompok terbang", "klw": "kalau", "km": "kamu", "kmps": "kampus", "kmrn": "kemarin", "knal": "kenal", "knp": "kenapa", "kodya": "kota madya", "komdis": "komisi disiplin", "komsov": "komunis sovyet", "kongkow": "kumpul bareng teman-teman", "kopdar": "kopi darat", "korup": "korupsi", "kpn": "kapan", "krenz": "keren", "krm": "kirim", "kt": "kita", "ktmu": "ketemu", "ktr": "kantor", "kuper": "kurang pergaulan", "kw": "imitasi", "kyk": "seperti", "la": "lah", "lam": "salam", "lamp": "lampiran", "lanud": "landasan udara", "latgab": "latihan gabungan", "lebay": "berlebihan", "leh": "boleh", "lelet": "lambat", "lemot": "lambat", "lgi": "lagi", "lgsg": "langsung", "liat": "lihat", "litbang": "penelitian dan pengembangan", "lmyn": "lumayan", "lo": "kamu", "loe": "kamu", "lola": "lambat berfikir", "louph": "cinta", "low": "kalau", "lp": "lupa", "luber": "langsung, umum, bebas, dan rahasia", "luchuw": "lucu", "lum": "belum", "luthu": "lucu", "lwn": "lawan", "maacih": "terima kasih", "mabal": "bolos", "macem": "macam", "macih": "masih", "maem": "makan", "magabut": "makan gaji buta", "maho": "homo", "mak jang": "kaget", "maksain": "memaksa", "malem": "malam", "mam": "makan", "maneh": "kamu", "maniez": "manis", "mao": "mau", "masukin": "masukkan", "melu": "ikut", "mepet": "dekat sekali", "mgu": "minggu", "migas": "minyak dan gas bumi", "mikol": "minuman beralkohol", "miras": "minuman keras", "mlah": "malah", "mngkn": "mungkin", "mo": "mau", "mokad": "mati", "moso": "masa", "mpe": "sampai", "msk": "masuk", "mslh": "masalah", "mt": "makan teman", "mubes": "musyawarah besar", "mulu": "melulu", "mumpung": "selagi", "munas": "musyawarah nasional", "muntaber": "muntah dan berak", "musti": "mesti", "muupz": "maaf", "mw": "now watching", "n": "dan", "nanam": "menanam", "nanya": "bertanya", "napa": "kenapa", "napi": "narapidana", "napza": "narkotika, alkohol, psikotropika, dan zat adiktif ", "narkoba": "narkotika, psikotropika, dan obat terlarang", "nasgor": "nasi goreng", "nda": "tidak", "ndiri": "sendiri", "ne": "ini", "nekolin": "neokolonialisme", "nembak": "menyatakan cinta", "ngabuburit": "menunggu berbuka puasa", "ngaku": "mengaku", "ngambil": "mengambil", "nganggur": "tidak punya pekerjaan", "ngapah": "kenapa", "ngaret": "terlambat", "ngasih": "memberikan", "ngebandel": "berbuat bandel", "ngegosip": "bergosip", "ngeklaim": "mengklaim", "ngeksis": "menjadi eksis", "ngeles": "berkilah", "ngelidur": "menggigau", "ngerampok": "merampok", "ngga": "tidak", "ngibul": "berbohong", "ngiler": "mau", "ngiri": "iri", "ngisiin": "mengisikan", "ngmng": "bicara", "ngomong": "bicara", "ngubek2": "mencari-cari", "ngurus": "mengurus", "nie": "ini", "nih": "ini", "niyh": "nih", "nmr": "nomor", "nntn": "nonton", "nobar": "nonton bareng", "np": "now playing", "ntar": "nanti", "ntn": "nonton", "numpuk": "bertumpuk", "nutupin": "menutupi", "nyari": "mencari", "nyekar": "menyekar", "nyicil": "mencicil", "nyoblos": "mencoblos", "nyokap": "ibu", "ogah": "tidak mau", "ol": "online", "ongkir": "ongkos kirim", "oot": "out of topic", "org2": "orang-orang", "ortu": "orang tua", "otda": "otonomi daerah", "otw": "on the way, sedang di jalan", "pacal": "pacar", "pake": "pakai", "pala": "kepala", "pansus": "panitia khusus", "parpol": "partai politik", "pasutri": "pasangan suami istri", "pd": "pada", "pede": "percaya diri", "pelatnas": "pemusatan latihan nasional", "pemda": "pemerintah daerah", "pemkot": "pemerintah kota", "pemred": "pemimpin redaksi", "penjas": "pendidikan jasmani", "perda": "peraturan daerah", "perhatiin": "perhatikan", "pesenan": "pesanan", "pgang": "pegang", "pi": "tapi", "pilkada": "pemilihan kepala daerah", "pisan": "sangat", "pk": "penjahat kelamin", "plg": "paling", "pmrnth": "pemerintah", "polantas": "polisi lalu lintas", "ponpes": "pondok pesantren", "pp": "pulang pergi", "prg": "pergi", "prnh": "pernah", "psen": "pesan", "pst": "pasti", "pswt": "pesawat", "pw": "posisi nyaman", "qmu": "kamu", "rakor": "rapat koordinasi", "ranmor": "kendaraan bermotor", "re": "reply", "ref": "referensi", "rehab": "rehabilitasi", "rempong": "sulit", "repp": "balas", "restik": "reserse narkotika", "rhs": "rahasia", "rmh": "rumah", "ru": "baru", "ruko": "rumah toko", "rusunawa": "rumah susun sewa", "ruz": "terus", "saia": "saya", "salting": "salah tingkah", "sampe": "sampai", "samsek": "sama sekali", "sapose": "siapa", "satpam": "satuan pengamanan", "sbb": "sebagai berikut", "sbh": "sebuah", "sbnrny": "sebenarnya", "scr": "secara", "sdgkn": "sedangkan", "sdkt": "sedikit", "se7": "setuju", "sebelas dua belas": "mirip", "sembako": "sembilan bahan pokok", "sempet": "sempat", "sendratari": "seni drama tari", "sgt": "sangat", "shg": "sehingga", "siech": "sih", "sikon": "situasi dan kondisi", "sinetron": "sinema elektronik", "siramin": "siramkan", "sj": "saja", "skalian": "sekalian", "sklh": "sekolah", "skt": "sakit", "slesai": "selesai", "sll": "selalu", "slma": "selama", "slsai": "selesai", "smpt": "sempat", "smw": "semua", "sndiri": "sendiri", "soljum": "sholat jumat", "songong": "sombong", "sory": "maaf", "sosek": "sosial-ekonomi", "sotoy": "sok tahu", "spa": "siapa", "sppa": "siapa", "spt": "seperti", "srtfkt": "sertifikat", "stiap": "setiap", "stlh": "setelah", "suk": "masuk", "sumpek": "sempit", "syg": "sayang", "t4": "tempat", "tajir": "kaya", "tau": "tahu", "taw": "tahu", "td": "tadi", "tdk": "tidak", "teh": "kakak perempuan", "telat": "terlambat", "telmi": "telat berpikir", "temen": "teman", "tengil": "menyebalkan", "tepar": "terkapar", "tggu": "tunggu", "tgu": "tunggu", "thankz": "terima kasih", "thn": "tahun", "tilang": "bukti pelanggaran", "tipiwan": "TvOne", "tks": "terima kasih", "tlp": "telepon", "tls": "tulis", "tmbah": "tambah", "tmen2": "teman-teman", "tmpah": "tumpah", "tmpt": "tempat", "tngu": "tunggu", "tnyta": "ternyata", "tokai": "tai", "toserba": "toko serba ada", "tpi": "tapi", "trdhulu": "terdahulu", "trima": "terima kasih", "trm": "terima", "trs": "terus", "trutama": "terutama", "ts": "penulis", "tst": "tahu sama tahu", "ttg": "tentang", "tuch": "tuh", "tuir": "tua", "tw": "tahu", "u": "kamu", "ud": "sudah", "udah": "sudah", "ujg": "ujung", "ul": "ulangan", "unyu": "lucu", "uplot": "unggah", "urang": "saya", "usah": "perlu", "utk": "untuk", "valas": "valuta asing", "w/": "dengan", "wadir": "wakil direktur", "wamil": "wajib militer", "warkop": "warung kopi", "warteg": "warung tegal", "wat": "buat", "wkt": "waktu", "wtf": "what the fuck", "xixixi": "tertawa", "ya": "iya", "yap": "iya", "yaudah": "ya sudah", "yawdah": "ya sudah", "yg": "yang", "yl": "yang lain", "yo": "iya", "yowes": "ya sudah", "yup": "iya", "7an": "tujuan", "ababil": "abg labil", "acc": "accord", "adlah": "adalah", "adoh": "aduh", "aha": "tertawa", "aing": "saya", "aja": "saja", "ajj": "saja", "aka": "dikenal juga sebagai", "akko": "aku", "akku": "aku", "akyu": "aku", "aljasa": "asal jadi saja", "ama": "sama", "ambl": "ambil", "anjir": "anjing", "ank": "anak", "ap": "apa", "apaan": "apa", "ape": "apa", "aplot": "unggah", "apva": "apa", "aqu": "aku", "asap": "sesegera mungkin", "aseek": "asyik", "asek": "asyik", "aseknya": "asyiknya", "asoy": "asyik", "astrojim": "astagfirullahaladzim", "ath": "kalau begitu", "atuh": "kalau begitu", "ava": "avatar", "aws": "awas", "ayang": "sayang", "ayok": "ayo", "bacot": "banyak bicara", "bales": "balas", "bangdes": "pembangunan desa", "bangkotan": "tua", "banpres": "bantuan presiden", "bansarkas": "bantuan sarana kesehatan", "bazis": "badan amal, zakat, infak, dan sedekah", "bcoz": "karena", "beb": "sayang", "bejibun": "banyak", "belom": "belum", "bener": "benar", "ber2": "berdua", "berdikari": "berdiri di atas kaki sendiri", "bet": "banget", "beti": "beda tipis", "beut": "banget", "bgd": "banget", "bgs": "bagus", "bhubu": "tidur", "bimbuluh": "bimbingan dan penyuluhan", "bisi": "kalau-kalau", "bkn": "bukan", "bl": "beli", "blg": "bilang", "blm": "belum", "bls": "balas", "bnchi": "benci", "bngung": "bingung", "bnyk": "banyak", "bohay": "badan aduhai", "bokep": "porno", "bokin": "pacar", "bole": "boleh", "bolot": "bodoh", "bonyok": "ayah ibu", "bpk": "bapak", "brb": "segera kembali", "brngkt": "berangkat", "brp": "berapa", "brur": "saudara laki-laki", "bsa": "bisa", "bsk": "besok", "bu_bu": "tidur", "bubarin": "bubarkan", "buber": "buka bersama", "bujubune": "luar biasa", "buser": "buru sergap", "bwhn": "bawahan", "byar": "bayar", "byr": "bayar", "c8": "chat", "cabut": "pergi", "caem": "cakep", "cama-cama": "sama-sama", "cangcut": "celana dalam", "cape": "capek", "caur": "jelek", "cekak": "tidak ada uang", "cekidot": "coba lihat", "cemplungin": "cemplungkan", "ceper": "pendek", "ceu": "kakak perempuan", "cewe": "cewek", "cibuk": "sibuk", "cin": "cinta", "ciye": "cie", "ckck": "ck", "clbk": "cinta lama bersemi kembali", "cmpr": "campur", "cnenk": "senang", "congor": "mulut", "cow": "cowok", "coz": "karena", "cpa": "siapa", "gokil": "gila", "gombal": "suka merayu", "gpl": "tidak pakai lama", "gpp": "tidak apa-apa", "gretong": "gratis", "gt": "begitu", "gtw": "tidak tahu", "gue": "saya", "guys": "teman-teman", "gws": "cepat sembuh", "haghaghag": "tertawa", "hakhak": "tertawa", "handak": "bahan peledak", "hansip": "pertahanan sipil", "hellow": "halo", "helow": "halo", "hi": "hai", "hlng": "hilang", "hnya": "hanya", "houm": "rumah", "hrs": "harus", "hubad": "hubungan angkatan darat", "hubla": "perhubungan laut", "huft": "mengeluh", "humas": "hubungan masyarakat", "idk": "saya tidak tahu", "ilfeel": "tidak suka", "imba": "jago sekali", "imoet": "imut", "info": "informasi", "itung": "hitung", "isengin": "bercanda", "iyala": "iya lah", "iyo": "iya", "jablay": "jarang dibelai", "jadul": "jaman dulu", "jancuk": "anjing", "jd": "jadi", "jdikan": "jadikan", "jg": "juga", "jgn": "jangan", "jijay": "jijik", "jkt": "jakarta", "jnj": "janji", "jth": "jatuh", "jurdil": "jujur adil", "jwb": "jawab", "ka": "kakak", "kabag": "kepala bagian", "kacian": "kasihan", "kadit": "kepala direktorat", "kaga": "tidak", "kaka": "kakak", "kamtib": "keamanan dan ketertiban", "kamuh": "kamu", "kamyu": "kamu", "kapt": "kapten", "kasat": "kepala satuan", "kasubbid": "kepala subbidang", "kau": "kamu", "kbar": "kabar", "kcian": "kasihan", "keburu": "terlanjur", "kedubes": "kedutaan besar", "kek": "seperti", "keknya": "kayaknya", "keliatan": "kelihatan", "keneh": "masih", "kepikiran": "terpikirkan", "kepo": "mau tahu urusan orang", "kere": "tidak punya uang", "kesian": "kasihan", "ketauan": "ketahuan", "keukeuh": "keras kepala", "khan": "kan", "kibus": "kaki busuk", "kk": "kakak", "klian": "kalian", "klo": "kalau", "kluarga": "keluarga", "klwrga": "keluarga", "kmari": "kemari", "kmpus": "kampus", "kn": "kan", "knl": "kenal", "knpa": "kenapa", "kog": "kok", "kompi": "komputer", "komtiong": "komunis Tiongkok", "konjen": "konsulat jenderal", "koq": "kok", "kpd": "kepada", "kptsan": "keputusan", "krik": "garing", "krn": "karena", "ktauan": "ketahuan", "ktny": "katanya", "kudu": "harus", "kuq": "kok", "ky": "seperti", "kykny": "kayanya", "laka": "kecelakaan", "lambreta": "lambat", "lansia": "lanjut usia", "lapas": "lembaga pemasyarakatan", "lbur": "libur", "lekong": "laki-laki", "lg": "lagi", "lgkp": "lengkap", "lht": "lihat", "linmas": "perlindungan masyarakat", "lmyan": "lumayan", "lngkp": "lengkap", "loch": "loh", "lol": "tertawa", "lom": "belum", "loupz": "cinta", "lowh": "kamu", "lu": "kamu", "luchu": "lucu", "luff": "cinta", "luph": "cinta", "lw": "kamu", "lwt": "lewat", "maaciw": "terima kasih", "mabes": "markas besar", "macem-macem": "macam-macam", "madesu": "masa depan suram", "maen": "main", "mahatma": "maju sehat bersama", "mak": "ibu", "makasih": "terima kasih", "malah": "bahkan", "malu2in": "memalukan", "mamz": "makan", "manies": "manis", "mantep": "mantap", "markus": "makelar kasus", "mba": "mbak", "mending": "lebih baik", "mgkn": "mungkin", "mhn": "mohon", "miker": "minuman keras", "milis": "mailing list", "mksd": "maksud", "mls": "malas", "mnt": "minta", "moge": "motor gede", "mokat": "mati", "mosok": "masa", "msh": "masih", "mskpn": "meskipun", "msng2": "masing-masing", "muahal": "mahal", "muker": "musyawarah kerja", "mumet": "pusing", "muna": "munafik", "munaslub": "musyawarah nasional luar biasa", "musda": "musyawarah daerah", "muup": "maaf", "muuv": "maaf", "nal": "kenal", "nangis": "menangis", "naon": "apa", "napol": "narapidana politik", "naq": "anak", "narsis": "bangga pada diri sendiri", "nax": "anak", "ndak": "tidak", "ndut": "gendut", "nekolim": "neokolonialisme", "nelfon": "menelepon", "ngabis2in": "menghabiskan", "ngakak": "tertawa", "ngambek": "marah", "ngampus": "pergi ke kampus", "ngantri": "mengantri", "ngapain": "sedang apa", "ngaruh": "berpengaruh", "ngawur": "berbicara sembarangan", "ngeceng": "kumpul bareng-bareng", "ngeh": "sadar", "ngekos": "tinggal di kos", "ngelamar": "melamar", "ngeliat": "melihat", "ngemeng": "bicara terus-terusan", "ngerti": "mengerti", "nggak": "tidak", "ngikut": "ikut", "nginep": "menginap", "ngisi": "mengisi", "ngmg": "bicara", "ngocol": "lucu", "ngomongin": "membicarakan", "ngumpul": "berkumpul", "ni": "ini", "nyasar": "tersesat", "nyariin": "mencari", "nyiapin": "mempersiapkan", "nyiram": "menyiram", "nyok": "ayo", "o/": "oleh", "ok": "ok", "priksa": "periksa", "pro": "profesional", "psn": "pesan", "psti": "pasti", "puanas": "panas", "qmo": "kamu", "qt": "kita", "rame": "ramai", "raskin": "rakyat miskin", "red": "redaksi", "reg": "register", "rejeki": "rezeki", "renstra": "rencana strategis", "reskrim": "reserse kriminal", "sni": "sini", "somse": "sombong sekali", "sorry": "maaf", "sosbud": "sosial-budaya", "sospol": "sosial-politik", "sowry": "maaf", "spd": "sepeda", "sprti": "seperti", "spy": "supaya", "stelah": "setelah", "subbag": "subbagian", "sumbangin": "sumbangkan", "sy": "saya", "syp": "siapa", "tabanas": "tabungan pembangunan nasional", "tar": "nanti", "taun": "tahun", "tawh": "tahu", "tdi": "tadi", "te2p": "tetap", "tekor": "rugi", "telkom": "telekomunikasi", "telp": "telepon", "temen2": "teman-teman", "tengok": "menjenguk", "terbitin": "terbitkan", "tgl": "tanggal", "thanks": "terima kasih", "thd": "terhadap", "thx": "terima kasih", "tipi": "TV", "tkg": "tukang", "tll": "terlalu", "tlpn": "telepon", "tman": "teman", "tmbh": "tambah", "tmn2": "teman-teman", "tmph": "tumpah", "tnda": "tanda", "tnh": "tanah", "togel": "toto gelap", "tp": "tapi", "tq": "terima kasih", "trgntg": "tergantung", "trims": "terima kasih", "cb": "coba", "y": "ya", "munfik": "munafik", "reklamuk": "reklamasi", "sma": "sama", "tren": "trend", "ngehe": "kesal", "mz": "mas", "analisise": "analisis", "sadaar": "sadar", "sept": "september", "nmenarik": "menarik", "zonk": "bodoh", "rights": "benar", "simiskin": "miskin", "ngumpet": "sembunyi", "hardcore": "keras", "akhirx": "akhirnya", "solve": "solusi", "watuk": "batuk", "ngebully": "intimidasi", "masy": "masyarakat", "still": "masih", "tauk": "tahu", "mbual": "bual", "tioghoa": "tionghoa", "ngentotin": "senggama", "kentot": "senggama", "faktakta": "fakta", "sohib": "teman", "rubahnn": "rubah", "trlalu": "terlalu", "nyela": "cela", "heters": "pembenci", "nyembah": "sembah", "most": "paling", "ikon": "lambang", "light": "terang", "pndukung": "pendukung", "setting": "atur", "seting": "akting", "next": "lanjut", "waspadalah": "waspada", "gantengsaya": "ganteng", "parte": "partai", "nyerang": "serang", "nipu": "tipu", "ktipu": "tipu", "jentelmen": "berani", "buangbuang": "buang", "tsangka": "tersangka", "kurng": "kurang", "ista": "nista", "less": "kurang", "koar": "teriak", "paranoid": "takut", "problem": "masalah", "tahi": "kotoran", "tirani": "tiran", "tilep": "tilap", "happy": "bahagia", "tak": "tidak", "penertiban": "tertib", "uasai": "kuasa", "mnolak": "tolak", "trending": "trend", "taik": "tahi", "wkwkkw": "tertawa", "ahokncc": "ahok", "istaa": "nista", "benarjujur": "jujur", "mgkin": "mungkin", "kikir" : "pelit"}

In [39]:
#testing model
import string
import re
import csv
import requests
import emoji
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
def cleaningTXT(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text)
  text = re.sub(r'#[A-Za-z0-9]+', '', text)
  text = re.sub(r'RT[\s]', '', text)
  text = re.sub(r"http\S+", '', text)
  text = re.sub(r'[0-9]+', '', text)
  text = emoji.replace_emoji(text, replace=' emoji ')
  text = re.sub(r'[^\w\s]', '', text)
  text = text.replace('\n', ' ')
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = ' '.join(text.strip().split())

  return text

def casefold(text):
  text = text.lower()
  return text

def tokenizeTXT(text):
  text = word_tokenize(text)
  return text

def filteringTXT(text):
  stopwordsID = set(stopwords.words('indonesian'))
  stopwordsEN = set(stopwords.words('english'))

  stopwordsID.update(stopwordsEN)
  stopwordsID.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy", "wkwk"])

  filtered = []

  for word in text:
    if word not in stopwordsID:
      filtered.append(word)
  text = filtered
  return text

def stemmingTXT(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  words = text.split()

  stem_word = [stemmer.stem(word) for word in words ]
  stem_text = ' '.join(stem_word)
  text = stem_text
  return text

def slangTXT(text):
  words = text.split()
  fix = []

  for word in words:
    if word.lower() in slangwords:
      fix.append(slangwords[word.lower()])
    else:
      fix.append(word)
  text = ' '.join(fix)
  return text

def sentenced(words):
  sentence = ' '.join(word for word in words)
  return sentence

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [65]:
trial = "genshin game baik"

step1 = cleaningTXT(trial)
step2 = casefold(step1)
step3 = slangTXT(step2)
step4 = tokenizeTXT(step3)
step5 = filteringTXT(step4)
step6 = sentenced(step5)
step7 =  [word for word in step6 if word in wordmodel.wv.key_to_index]

vectors = [wordmodel.wv[word] for word in step7]
sentence_vector = np.mean(vectors, axis=0)

sentimen = rf_model_w2v.predict([sentence_vector])
if sentimen == 'positive':
    print("positive")
else:
    print("negative")

positive


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
